# Análisis de Datos del Baloto

**Objetivo**: Aplicar técnicas de limpieza, visualización y análisis de datos a los resultados históricos del Baloto.

**Dataset**: Resultados históricos desde 2017 hasta 2023 con columnas: Date, C1, C2, C3, C4, C5, SB

# IMPORTE DE LIBRERIAS

In [1]:
#IMPORTE DE LIBRERIAS

import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import pandas as pd
from Pad_dataSet_Act_4 import Pad_dataSet
import numpy as np
import kagglehub
import seaborn as sns
import os
import zipfile
import tkinter
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.backends.backend_pdf import PdfPages
from IPython.display import display, Markdown


# CARGAR DATOS

In [2]:
df = pd.read_csv('baloto.csv')  # Asegúrate que este archivo está en la misma carpeta
df.head()

,Date,C1,C2,C3,C4,C5,SB
0,4/22/2017,15,28,36,37,41,10
1,4/26/2017,1,2,25,31,33,14
2,4/29/2017,6,19,25,28,36,1
3,5/3/2017,5,6,7,12,37,6
4,5/6/2017,1,20,36,39,43,10


# Limpieza de datos

In [3]:
# Convertir la columna de fecha
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Eliminar duplicados y nulos
df = df.drop_duplicates()
df = df.dropna()

# Convertir columnas numéricas a enteros
numeros_cols = ['C1', 'C2', 'C3', 'C4', 'C5', 'SB']
df[numeros_cols] = df[numeros_cols].apply(pd.to_numeric, errors='coerce')

# Validar que los números estén entre 1 y 45
for col in numeros_cols:
    df = df[(df[col] >= 1) & (df[col] <= 45)]

df.reset_index(drop=True, inplace=True)
df.head()

,Date,C1,C2,C3,C4,C5,SB
0,2017-04-22,15,28,36,37,41,10
1,2017-04-26,1,2,25,31,33,14
2,2017-04-29,6,19,25,28,36,1
3,2017-05-03,5,6,7,12,37,6
4,2017-05-06,1,20,36,39,43,10


## VISUALIZACION Y ANALISIS DE LOS DATOS OBTENIDOS

Porcentaje de aparición de cada número

In [4]:
# Unificar todos los números en una sola lista
numeros = df[numeros_cols].values.flatten()

# Calcular frecuencia y porcentaje
frecuencia = pd.Series(numeros).value_counts().sort_index()
porcentaje = (frecuencia / frecuencia.sum()) * 100

# Gráfico
plt.figure(figsize=(12, 6))
sns.barplot(x=porcentaje.index, y=porcentaje.values, palette="mako")
plt.title("Porcentaje de aparición de cada número")
plt.xlabel("Número")
plt.ylabel("Porcentaje (%)")
plt.xticks(rotation=90)
plt.show()

C:\Users\DAVID GOMEZ\AppData\Local\Temp\ipykernel_13228\4176346397.py:10: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=porcentaje.index, y=porcentaje.values, palette="mako")


Análisis de Pares e Impares

In [5]:
def contar_pares_impares(fila):
    numeros = fila[numeros_cols]
    pares = sum(n % 2 == 0 for n in numeros)
    impares = len(numeros) - pares
    return pd.Series({'PARES': pares, 'IMPARES': impares})

df[['PARES', 'IMPARES']] = df.apply(contar_pares_impares, axis=1)

# Histograma de pares
sns.histplot(df['PARES'], bins=range(0, 7), kde=False, color='skyblue')
plt.title("Distribución de números pares por sorteo")
plt.xlabel("Cantidad de pares")
plt.ylabel("Frecuencia")
plt.show()


Análisis de números consecutivos

In [6]:
def contar_consecutivos(fila):
    numeros = sorted(fila[numeros_cols])
    consecutivos = sum(1 for i in range(len(numeros)-1) if numeros[i+1] - numeros[i] == 1)
    return consecutivos

df['CONSECUTIVOS'] = df.apply(contar_consecutivos, axis=1)

sns.countplot(x='CONSECUTIVOS', data=df, palette='coolwarm')
plt.title("Cantidad de números consecutivos por sorteo")
plt.xlabel("Consecutivos")
plt.ylabel("Frecuencia")
plt.show()


C:\Users\DAVID GOMEZ\AppData\Local\Temp\ipykernel_13228\2281173013.py:8: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(x='CONSECUTIVOS', data=df, palette='coolwarm')


Análisis por rangos

In [7]:
def clasificar_rango(n):
    if n <= 15:
        return "1-15"
    elif n <= 30:
        return "16-30"
    else:
        return "31-45"

rango_series = pd.Series(numeros).dropna().astype(int).map(clasificar_rango)
rango_frecuencia = rango_series.value_counts(normalize=True) * 100

rango_frecuencia.plot(kind='bar', color='coral')
plt.title("Porcentaje de aparición por rango numérico")
plt.ylabel("Porcentaje (%)")
plt.xlabel("Rango")
plt.show()


## GENERAR REPORTE EN PDF

In [8]:
df = pd.read_csv("baloto.csv")
print("Primeras filas del dataframe:")
print(df.head())
print("\nColumnas del dataframe:")
print(df.columns)

# Limpieza de datos
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.drop_duplicates()
df = df.dropna()

# Convertir columnas numéricas
numeros_columns = ['C1', 'C2', 'C3', 'C4', 'C5', 'SB']
df[numeros_columns] = df[numeros_columns].apply(pd.to_numeric, errors='coerce')

# Filtrar valores válidos (1-45 para números principales, 1-16 para SB)
for col in ['C1', 'C2', 'C3', 'C4', 'C5']:
    df = df[(df[col] >= 1) & (df[col] <= 45)]
df = df[(df['SB'] >= 1) & (df['SB'] <= 16)]

# Análisis de frecuencia
numeros_principales = df[['C1', 'C2', 'C3', 'C4', 'C5']].values.flatten()
frecuencia_principales = pd.Series(numeros_principales).value_counts().sort_index()
porcentaje_principales = (frecuencia_principales / frecuencia_principales.sum()) * 100

frecuencia_sb = df['SB'].value_counts().sort_index()
porcentaje_sb = (frecuencia_sb / frecuencia_sb.sum()) * 100

# Análisis de pares e impares
def contar_pares_impares(fila):
    numeros = fila[['C1', 'C2', 'C3', 'C4', 'C5']]
    pares = sum(n % 2 == 0 for n in numeros)
    return pd.Series({'PARES': pares, 'IMPARES': 5 - pares})

df[['PARES', 'IMPARES']] = df.apply(contar_pares_impares, axis=1)

# Números consecutivos
def contar_consecutivos(fila):
    numeros = sorted(fila[['C1', 'C2', 'C3', 'C4', 'C5']])
    consecutivos = sum(1 for i in range(len(numeros)-1) if numeros[i+1] - numeros[i] == 1)
    return consecutivos

df['CONSECUTIVOS'] = df.apply(contar_consecutivos, axis=1)

# Rangos numéricos
def clasificar_rangos(num):
    if num <= 15:
        return "1-15"
    elif num <= 30:
        return "16-30"
    else:
        return "31-45"

rango_series = pd.Series(numeros_principales).dropna().astype(int).map(clasificar_rangos)
rango_frecuencia = rango_series.value_counts(normalize=True) * 100

# Crear el PDF
pdf_path = "Analisis_Baloto_Resultados.pdf"
with PdfPages(pdf_path) as pdf:
    
    # Gráfico 1: Frecuencia números principales
    plt.figure(figsize=(14, 6))
    sns.barplot(x=frecuencia_principales.index, y=frecuencia_principales.values, palette="viridis")
    plt.title("Frecuencia de aparición de números principales (1-45)")
    plt.xlabel("Número")
    plt.ylabel("Veces que ha salido")
    plt.xticks(rotation=90)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    pdf.savefig()
    plt.close()
    
    # Gráfico 2: Frecuencia Super Balota
    plt.figure(figsize=(10, 5))
    sns.barplot(x=frecuencia_sb.index, y=frecuencia_sb.values, palette="rocket")
    plt.title("Frecuencia de aparición de Super Balota (1-16)")
    plt.xlabel("Número de Super Balota")
    plt.ylabel("Veces que ha salido")
    pdf.savefig()
    plt.close()
    
    # Gráfico 3: Distribución pares/impares
    plt.figure(figsize=(10, 5))
    df['PARES'].value_counts().sort_index().plot(kind='bar', color=['skyblue', 'lightgreen', 'salmon'])
    plt.title("Distribución de cantidad de números pares por sorteo")
    plt.xlabel("Cantidad de pares (de 5 números)")
    plt.ylabel("Número de sorteos")
    pdf.savefig()
    plt.close()
    
    # Gráfico 4: Números consecutivos
    plt.figure(figsize=(10, 5))
    sns.countplot(x='CONSECUTIVOS', data=df, palette='coolwarm')
    plt.title("Cantidad de números consecutivos por sorteo")
    plt.xlabel("Números consecutivos")
    plt.ylabel("Número de sorteos")
    pdf.savefig()
    plt.close()
    
    # Gráfico 5: Rangos numéricos
    plt.figure(figsize=(8, 5))
    rango_frecuencia.plot(kind='bar', color=['#FF9999', '#66B2FF', '#99FF99'])
    plt.title("Distribución por rangos numéricos (números principales)")
    plt.ylabel("Porcentaje (%)")
    plt.xlabel("Rango")
    pdf.savefig()
    plt.close()
    
    # Página de conclusiones
    fig = plt.figure(figsize=(11, 8.5))
    plt.axis('off')
    conclusiones = """
     CONCLUSIONES DEL ANÁLISIS DEL BALOTO
    
    • Se analizaron {} sorteos históricos del Baloto.
    
    • Los números principales (1-45) muestran las siguientes frecuencias:
      - Número más frecuente: {}
      - Número menos frecuente: {}
    
    • La Super Balota (1-16) muestra:
      - Número más frecuente: {}
      - Número menos frecuente: {}
    
    • Distribución de pares/impares:
      - Combinación más común: {} pares y {} impares
    
    • En {:.1f}% de los sorteos aparecen al menos 2 números consecutivos.
    
    • Distribución por rangos:
      - 1-15: {:.1f}%
      - 16-30: {:.1f}%
      - 31-45: {:.1f}%
    """.format(
        len(df),
        frecuencia_principales.idxmax(), frecuencia_principales.idxmin(),
        frecuencia_sb.idxmax(), frecuencia_sb.idxmin(),
        df['PARES'].mode()[0], 5 - df['PARES'].mode()[0],
        (df['CONSECUTIVOS'] >= 2).mean() * 100,
        rango_frecuencia.get("1-15", 0),
        rango_frecuencia.get("16-30", 0),
        rango_frecuencia.get("31-45", 0)
    )
    plt.text(0.05, 0.5, conclusiones, fontsize=12, va="center")
    pdf.savefig(fig)
    plt.close()

print(f"Análisis completado. El reporte se ha guardado en: {pdf_path}")

Primeras filas del dataframe:
        Date  C1  C2  C3  C4  C5  SB
0  4/22/2017  15  28  36  37  41  10
1  4/26/2017   1   2  25  31  33  14
2  4/29/2017   6  19  25  28  36   1
3   5/3/2017   5   6   7  12  37   6
4   5/6/2017   1  20  36  39  43  10

Columnas del dataframe:
Index(['Date', 'C1', 'C2', 'C3', 'C4', 'C5', 'SB'], dtype='object')


C:\Users\DAVID GOMEZ\AppData\Local\Temp\ipykernel_13228\1873827381.py:63: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=frecuencia_principales.index, y=frecuencia_principales.values, palette="viridis")
C:\Users\DAVID GOMEZ\AppData\Local\Temp\ipykernel_13228\1873827381.py:74: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=frecuencia_sb.index, y=frecuencia_sb.values, palette="rocket")
C:\Users\DAVID GOMEZ\AppData\Local\Temp\ipykernel_13228\1873827381.py:92: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(x='CONSECUTIVOS', data=df, palette='

Análisis completado. El reporte se ha guardado en: Analisis_Baloto_Resultados.pdf
